<a href="https://colab.research.google.com/github/Abdulrahman2k/databricks/blob/master/Apache_Spark_Linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Install Java, Spark, and Findspark**
This installs Apache Spark 2.3.1, Java 8, and Findspark, a library that makes it easy for Python to find Spark.

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget http://www-us.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz   
!tar xf spark-3.0.0-bin-hadoop3.2.tgz   
!pip install -q findspark

!ls

# **Set Environment Variables**
Set the locations where Spark and Java are installed.

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

# **Start a SparkSession**
This will start a local Spark session.

In [ ]:
import findspark
findspark.init()

from pyspark import SparkContext 
from pyspark import SparkConf 
from pyspark.sql import SQLContext

conf = SparkConf().setMaster("local[*]").setAppName('pyspark_Linear_Regression') 
sc = SparkContext(conf=conf) 
sqc = SQLContext(sc)

## **Read from local Machine**

In [ ]:
from google.colab import files
uploaded = files.upload()